In [ ]:
# Fungsi keanggotaan umum
def fungsi_segitiga(x, a, b, c):
    return max(0, min((x - a) / (b - a), (c - x) / (c - b)))

def fungsi_trapesium(x, a, b, c, d):
    """Fungsi keanggotaan bentuk trapesium dengan perlindungan pembagian nol."""
    left = 0
    right = 0
    if b != a:
        left = (x - a) / (b - a)
    else:
        left = 1 if x >= a else 0
    if d != c:
        right = (d - x) / (d - c)
    else:
        right = 1 if x <= d else 0
    return max(0, min(left, 1, right))

In [ ]:
# Fuzifikasi untuk setiap variabel input
def fuzzify_age(age_val):
    return {
        'Muda': fungsi_trapesium(age_val, 0, 0, 30, 40),
        'ParuhBaya': fungsi_segitiga(age_val, 30, 45, 60),
        'Tua': fungsi_trapesium(age_val, 50, 65, 100, 100)
    }

def fuzzify_glucose(glucose_val):
    return {
        'Normal': fungsi_trapesium(glucose_val, 50, 50, 100, 120),
        'Tinggi': fungsi_trapesium(glucose_val, 100, 140, 280, 280)
    }

def fuzzify_bmi(bmi_val):
    return {
        'Normal': fungsi_trapesium(bmi_val, 18, 18, 25, 30),
        'Berlebih': fungsi_trapesium(bmi_val, 25, 30, 50, 50)
    }

In [53]:
def defuzzify_sugeno(w_list, z_list):
    """
    Menghitung output akhir menggunakan metode Rata-rata Terbobot.
    """
    # Cek jika tidak ada aturan yang aktif
    if not w_list or sum(w_list) == 0:
        return 0
    
    # Hitung pembilang (numerator) dan penyebut (denominator)
    numerator = sum(w * z for w, z in zip(w_list, z_list))
    denominator = sum(w_list)
    
    return numerator / denominator

In [ ]:
def apply_rules_sugeno(age_fuzzy, glucose_fuzzy, bmi_fuzzy):
    rules = [
        # Risiko sangat tinggi
        ('Tua', 'Tinggi', 'Berlebih', 1.0),
        ('Tua', 'Tinggi', 'Normal', 0.9),
        ('Tua', 'Normal', 'Berlebih', 0.8),
        ('ParuhBaya', 'Tinggi', 'Berlebih', 0.8),
        # Risiko tinggi
        ('Tua', 'Normal', 'Normal', 0.7),
        ('ParuhBaya', 'Tinggi', 'Normal', 0.7),
        ('ParuhBaya', 'Normal', 'Berlebih', 0.7),
        ('Muda', 'Tinggi', 'Berlebih', 0.6),
        # Risiko sedang
        ('Tua', 'Tinggi', 'Berlebih', 1.0),  
        ('ParuhBaya', 'Tinggi', 'Berlebih', 0.8), 
        ('ParuhBaya', 'Normal', 'Normal', 0.5),
        ('Muda', 'Tinggi', 'Normal', 0.5),
        ('Muda', 'Normal', 'Berlebih', 0.4),
        # Risiko rendah
        ('Tua', 'Normal', 'Normal', 0.3),
        ('ParuhBaya', 'Normal', 'Normal', 0.2),
        ('Muda', 'Normal', 'Normal', 0.1),
        # Aturan tambahan untuk kombinasi ekstrem
        ('Muda', 'Tinggi', 'Berlebih', 0.6),
        ('Tua', 'Tinggi', 'Berlebih', 1.0),
        ('Tua', 'Tinggi', 'Normal', 0.9),
        ('Tua', 'Normal', 'Berlebih', 0.8),
        ('Tua', 'Normal', 'Normal', 0.7),
        ('ParuhBaya', 'Tinggi', 'Berlebih', 0.8),
        ('ParuhBaya', 'Tinggi', 'Normal', 0.7),
        ('ParuhBaya', 'Normal', 'Berlebih', 0.7),
        ('ParuhBaya', 'Normal', 'Normal', 0.5),
        ('Muda', 'Tinggi', 'Berlebih', 0.6),
        ('Muda', 'Tinggi', 'Normal', 0.5),
        ('Muda', 'Normal', 'Berlebih', 0.4),
        ('Muda', 'Normal', 'Normal', 0.1),
    ]
    w_list = []
    z_list = []
    for age_label, glucose_label, bmi_label, z in rules:
        w = min(age_fuzzy[age_label], glucose_fuzzy[glucose_label], bmi_fuzzy[bmi_label])
        if w > 0:
            w_list.append(w)
            z_list.append(z)
    return w_list, z_list

In [ ]:
def predict_stroke_risk_sugeno(input_data):
    """
    Fungsi utama untuk menjalankan seluruh proses Fuzzy Sugeno.
    `input_data` adalah dictionary, contoh: {'age': 67, 'avg_glucose_level': 228, 'bmi': 36}
    """
    # Fuzifikasi: Mengubah input crisp menjadi derajat keanggotaan fuzzy
    age_fuzzy = fuzzify_age(input_data['age'])
    glucose_fuzzy = fuzzify_glucose(input_data['avg_glucose_level'])
    bmi_fuzzy = fuzzify_bmi(input_data['bmi'])

    # Penerapan Aturan Fuzzy Sugeno
    w_values, z_values = apply_rules_sugeno(age_fuzzy, glucose_fuzzy, bmi_fuzzy)
    
    # Defuzifikasi
    final_risk_score = defuzzify_sugeno(w_values, z_values)
    
    return final_risk_score

# Uji Evaluasi Model Mamdani

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# Load dataset
df = pd.read_csv('stroke_preprocessed_for_fuzzy.csv')

# Prediksi skor risiko Sugeno untuk seluruh data
def predict_row(row):
    input_data = {
        'age': row['age'],
        'avg_glucose_level': row['avg_glucose_level'],
        'bmi': row['bmi']
    }
    return predict_stroke_risk_sugeno(input_data)
df['sugeno_risk_score'] = df.apply(predict_row, axis=1)

# Konversi skor ke label prediksi dengan threshold 0.5
df['sugeno_pred'] = (df['sugeno_risk_score'] >= 0.5).astype(int)

# Mengambil label target dari dataset sebenarnya dan hasil prediksi
y_true = df['stroke']
y_pred = df['sugeno_pred']

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print("=== Evaluasi Fuzzy Sugeno ===")
print(f"Akurasi  : {acc:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"Precision Sugeno: {precision:.3f}")
print(f"Recall Sugeno: {recall:.3f}")



=== Evaluasi Fuzzy Sugeno ===
Akurasi  : 0.4424
F1-score : 0.1437
Precision Sugeno: 0.078
Recall Sugeno: 0.960
